In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.stats import uniform
from BFM.MCMC import Gibbs_sampling

In [ ]:
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
P = 1000
K = 5
N = 500



In [ ]:
B_0 = np.random.binomial(1,1/3,(P,K)) * np.random.rand(P,K)
sigma2_0 = np.diag(uniform.rvs(loc = 0.1, scale = 0.9, size = P))
cov_0 = B_0 @ B_0.T + sigma2_0


In [ ]:
X = np.random.multivariate_normal(np.zeros(P), cov_0, N)

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(cov_0)
plt.colorbar()
plt.title("Covariance")
plt.show()

In [ ]:
B_sample, sigma2_sample = Gibbs_sampling(torch.from_numpy(X).to(device1), 100)

In [ ]:
A = torch.einsum('bij,bjk->bik',B_sample, B_sample.transpose(1,2))
for i in range(10000):
    A[i] = A[i] + torch.diag(sigma2_sample[i])

In [ ]:
A_mean = A.mean(0)

In [ ]:
plt.figure(figsize=(8, 6))
plt.imshow(A_mean)
plt.colorbar()
plt.title("Covariance")
plt.show()

In [ ]:
cov_true = torch.from_numpy(cov_0)

In [ ]:
(cov_true - A_mean).square().sum()/P**2

In [ ]:
(cov_true - A_mean).abs().sum()/P**2

In [ ]:
B_mean = B_sample.mean(0)

In [ ]:
for i in range(50):
    
    print(B_mean[:,i].abs().sum())